# Numerical Solvers

This notebook explains how to use the numerical solvers of the masters thesis "Fast, Accurate, and Scalable Numerical Wave Propagation: Enhancement by Deep Learning" by Luis Kaiser, supervised by Prof. Tsai (University of Texas Austin) and Prof. Klingenberg (University of Wuerzburg), in practice.

First, install all necessary libraries specified in `requirements.txt` using `pip` or `pip3` depending on your setup by running the command below.

In [ ]:
!pip3 install --upgrade pip
!pip3 install -r requirements.txt

Generally, the velocity Verlet algorithm and the RK4 pseudo-spectral algorithm are used to advance a two-dimensional wave field in a complex medium. By running the code below, we deploy one of the solvers for 10 time steps starting with an initial Gaussian pulse. This setup solves the wave equation with periodic boundary conditions. For absorbing boundary conditions, small modifications have to be made: First use the `velocity_verlet_tensor` implementation and use the parameter `boundary_c = absorbing`. Then, change the inputs `u, ut` to the function `velocity_verlet_tensor` to `torch.Tensor`.

In [ ]:
from numerical_solvers import get_velocity_model, pseudo_spectral_tensor, velocity_verlet_tensor, init_pulse_gaussian, WaveEnergyField_tensor
import matplotlib.pyplot as plt
import torch

def visualize_numerical_solver_periodic(
        vel_data_path = "data/crop_test.npz",
        method = "pseudo-spectral",
        dx = 2./128.,
        dt = 1/600.,
        dt_star = .06
):
    '''
    Parameters
    ----------
    vel_data_path : (string) path to velocity profile crops
    method : (string) "pseudo-spectral" or "velocity-verlet"
    res : (int) dimensionality of the input
    dx : (float) spatial step size numerical solver
    dt : (float) temporal step size numerical solver
    dt_star : (float) time interval the solver is applied once

    Returns
    -------
    10 advancements of timestep dt_star with periodic boundary conditions
    '''

    print(f"Advancement of two-dimensional wave fields using {method}:")

    vel = torch.from_numpy(get_velocity_model(vel_data_path))

    # computing initial condition using gaussian pulse (switch to pytorch tensor if needed)
    u, ut = init_pulse_gaussian(7000, 128, 0, 0)
    u, ut = torch.from_numpy(u), torch.from_numpy(ut)

    for s in range(10):

        # run one iteration of the RK4 / velocity Verlet method for time dt_star and step size dx, time increment dt
        if method == "pseudo-spectral":
            u, ut = pseudo_spectral_tensor(u, ut, vel, dx, dt, dt_star)
        else:  # method == "velocity_verlet"
            u, ut = velocity_verlet_tensor(u, ut, vel, dx, dt, dt_star)

        # change representation to energy semi-norm
        w = WaveEnergyField_tensor(u,ut,vel, dx)

        # visualize results
        plt.axis("off")
        plt.imshow(w)
        plt.title(f"wave field for iteration {s}")
        plt.show()

In [ ]:
visualize_numerical_solver_periodic()